In [ ]:
import xarray as xr
import numpy as np
import hydro_metrics as hym
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib as mpl
from scipy.stats import gaussian_kde
import matplotlib.ticker as ticker
import geopandas as gpd

In [ ]:
#wrf
#
points_wrf=xr.open_dataset(filepath+'\d04_raw_2018-wrf.nc')

points_nosa_predwrf=xr.open_dataset(filepath+'\d04_nosa_2018-wrf.nc')
points_sa_predwrf=xr.open_dataset(filepath+'\d04_sa_2018-wrf.nc')

#gpm
points_gpm=xr.open_dataset(filepath+'\d04_raw_2018-gpm.nc')
points_nosa_predgpm=xr.open_dataset(filepath+'\d04_nosa_2018-gpm.nc')
points_sa_predgpm=xr.open_dataset(filepath+'\d04_sa_2018-gpm.nc')

#era5
points_era5=xr.open_dataset(filepath+'\d04_raw_2018-era5.nc')
points_nosa_predera5=xr.open_dataset(filepath+'\d04_nosa_2018-era5.nc')
points_sa_predera5=xr.open_dataset(filepath+'\d04_sa_2018-era5.nc')

#gauge
ds_gauge=xr.open_dataset(filepath+'\d04_rain_gauge.nc')
ds_gauge=ds_gauge.sel(time=slice('2018-06-01T00:00:00','2018-09-30T23:00:00'))


In [ ]:
def plot_scatter(sim,obs,ax):
    ub=50
    unit='mm/d'
    
    xy = np.vstack([obs,sim])
    z = gaussian_kde(xy)(xy)
    # ax.scatter(obs,sim,color='blue',alpha=0.1,s=5)
    scatter=ax.scatter(obs, sim, c=z, s=3, cmap='jet', edgecolor='none',vmin=0,vmax=0.01) 


    ax.plot([0,ub], [0,ub], color='k',linewidth=0.5) # plot a  line with slope 1 and intercept 0
    ax.set_xlim([0,ub])
    ax.set_ylim([0,ub])

    return scatter

def xr_to_1d(obs,sim):
    ts=0
    obs_1d=obs.tp.values.reshape(-1).squeeze()
    mask = obs_1d>=ts

    obs_1d=obs_1d[mask]
    sim_1d=sim.tp.values.reshape(-1).squeeze()
    sim_1d=sim_1d[mask]
    
    return obs_1d,sim_1d

def ds_plot(ds_obs,ds_sim,ax):

    ds_obs_1d,ds_sim_1d=xr_to_1d(ds_obs,ds_sim)
    
    scatter=plot_scatter(ds_sim_1d,ds_obs_1d,ax)
    return scatter
    
params={'font.family':'serif',
	
    'font.serif':'Arial',
    'font.style':'normal', #normal
    'font.weight':'normal',#or'bold,
    'font.size':8,

    # 'ytick.major.left': True,
    "ytick.direction": "in",  # direction: in, out, or inout
    "xtick.direction": "in",  # direction: in, out, or inout
            }

mpl.rcParams['lines.linewidth'] = 1.5
mpl.rcParams['axes.linewidth']=0.75   #axes width
mpl.rcParams['xtick.labelsize'] = 7
mpl.rcParams['ytick.labelsize'] = 7

rcParams.update(params)

pos_l,pos_r=0.05,0.95
fig = plt.figure(figsize=(4.5,4),
                 )

#WRF
xlabel=''
ax1 = fig.add_subplot(3, 3,1, projection='scatter_density')
ax1.text(pos_l, pos_r, '(a)', transform=ax1.transAxes, fontsize=8, verticalalignment='top')
scatter=ds_plot(ds_gauge,points_wrf,ax1)

ax1.text(-0.38, 0.5, 'WRF-9km', transform=ax1.transAxes, 
        ha='center', va='center', rotation=90,fontsize=9,fontweight='bold')

ax1.set_ylabel('Estimates (mm/d)')

ax1.set_title('Raw',fontsize=9,fontweight='bold')

####
ax = fig.add_subplot(3, 3,2, projection='scatter_density')

ax.text(pos_l, pos_r, '(b)', transform=ax.transAxes, fontsize=8, verticalalignment='top')
scatter=ds_plot(ds_gauge,points_nosa_predwrf,ax)

ax.set_title('Baseline',fontsize=9,fontweight='bold')


xlabel=''
ax = fig.add_subplot(3, 3,3, projection='scatter_density')

ax.text(pos_l, pos_r, '(c)', transform=ax.transAxes, fontsize=8, verticalalignment='top')
scatter=ds_plot(ds_gauge,points_sa_predwrf,ax)

ax.set_title('HRDNet',fontsize=9,fontweight='bold')

# ax.set_xlabel('Gauge (mm/d)')
# ax.set_ylabel('mm/d')

#ERA5
xlabel=''
ax2 = fig.add_subplot(3, 3,4, projection='scatter_density')
scatter=ds_plot(ds_gauge,points_era5,ax2) #hour  
ax2.text(pos_l, pos_r, '(d)', transform=ax2.transAxes, fontsize=8, verticalalignment='top')

ax2.text(-0.38, 0.5, 'ERA5-Land', transform=ax2.transAxes, 
        ha='center', va='center', rotation=90,fontsize=9,fontweight='bold')

ax2.set_ylabel('Estimates (mm/d)')

####
xlabel=''
ax = fig.add_subplot(3, 3,5, projection='scatter_density')

ax.text(pos_l, pos_r, '(e)', transform=ax.transAxes, fontsize=8, verticalalignment='top')
scatter=ds_plot(ds_gauge,points_nosa_predera5,ax)

####
xlabel=''
ax = fig.add_subplot(3, 3,6, projection='scatter_density')

ax.text(pos_l, pos_r, '(f)', transform=ax.transAxes, fontsize=8, verticalalignment='top')
scatter=ds_plot(ds_gauge,points_sa_predera5,ax)

#GPM
xlabel=''

ax3 = fig.add_subplot(3, 3,7, projection='scatter_density')
scatter=ds_plot(ds_gauge,points_gpm,ax3)

ax3.text(pos_l, pos_r, '(g)', transform=ax3.transAxes, fontsize=8, verticalalignment='top')

ax3.text(-0.38, 0.5, 'IMERG', transform=ax3.transAxes, 
        ha='center', va='center', rotation=90,fontsize=9,fontweight='bold')

ax3.set_xlabel('Gauge (mm/d)')
ax3.set_ylabel('Estimates (mm/d)')

####
xlabel=''
ax = fig.add_subplot(3, 3,8, projection='scatter_density')

ax.text(pos_l, pos_r, '(h)', transform=ax.transAxes, fontsize=8, verticalalignment='top')
scatter=ds_plot(ds_gauge,points_nosa_predgpm,ax)
ax.set_xlabel('Gauge (mm/d)')
#
xlabel=''
ax = fig.add_subplot(3, 3,9, projection='scatter_density')

ax.text(pos_l, pos_r, '(i)', transform=ax.transAxes, fontsize=8, verticalalignment='top')
scatter=ds_plot(ds_gauge,points_sa_predgpm,ax)

cax = plt.axes([0.93, 0.15, 0.02, 0.68])  

cbar = plt.colorbar(scatter, cax=cax, orientation='vertical')

cbar.ax.set_title('Density', pad=5,fontsize=7)

ax.set_xlabel('Gauge (mm/d)')

plt.subplots_adjust(wspace=0.25,hspace=0.25)

plt.show()
